# Competition Objective is to detect fraud in transactions; 

## Data


In this competition you are predicting the probability that an online transaction is fraudulent, as denoted by the binary target ```isFraud```.

The data is broken into two files **identity** and **transaction**, which are joined by ```TransactionID```. 

> Note: Not all transactions have corresponding identity information.

**Transaction variables**

- TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
- TransactionAMT: transaction payment amount in USD
- ProductCD: product code, the product for each transaction
- card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
- addr: address
- dist: distance
- P_ and (R__) emaildomain: purchaser and recipient email domain
- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
- D1-D15: timedelta, such as days between previous transaction, etc.
- M1-M9: match, such as names on card and address, etc.
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.

**Categorical Features - Transaction**

- ProductCD
- emaildomain
- card1 - card6
- addr1, addr2
- P_emaildomain
- R_emaildomain
- M1 - M9

**Categorical Features - Identity**

- DeviceType
- DeviceInfo
- id_12 - id_38

**The TransactionDT feature is a timedelta from a given reference datetime (not an actual timestamp).**


# 1. Importation and memory reduction
## 1.1. Importing necessary libraries

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Standard plotly imports
#import plotly.plotly as py
import plotly.graph_objs as go

import plotly.express as px
import plotly.tools as tls
from plotly.subplots import make_subplots

from plotly.offline import iplot, init_notebook_mode
#import cufflinks
#import cufflinks as cf
import plotly.figure_factory as ff


# Using plotly + cufflinks in offline mode
init_notebook_mode(connected=True)
#cufflinks.go_offline(connected=True)

# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from xgboost import XGBClassifier
import xgboost as xgb

## Hyperopt modules
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING
from functools import partial

import os
import gc
import time

In [0]:
### Import data from google drive

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link1 = 'https://drive.google.com/open?id=1VLhLtqXYRNVtQbNaQr4M8TvtqVrJ6IDi'
link2 = 'https://drive.google.com/open?id=16BltOWr3AW_RYIWsHJJHDPRT5spX7h9H'
link3 = 'https://drive.google.com/open?id=18F3oYUH4AxKisji8YX0CeLaoNiIvL8K9'
link4 = 'https://drive.google.com/open?id=1JMa86BsJ2xfhgwgCQFrxmBwAHDQ4BvWi'
link5 = 'https://drive.google.com/open?id=1v5in81M0aYl-pX5u-8Tq4G8kwFp4CVez'
_, id1 = link1.split('=')
_, id2 = link2.split('=')
_, id3 = link3.split('=')
_, id4 = link4.split('=')
_, id5 = link5.split('=')

downloaded1 = drive.CreateFile({'id':id1}) 
downloaded1.GetContentFile('train_transaction.csv')  
downloaded2 = drive.CreateFile({'id':id2}) 
downloaded2.GetContentFile('train_identity.csv')  
downloaded3 = drive.CreateFile({'id':id3}) 
downloaded3.GetContentFile('test_transaction.csv')  
downloaded4 = drive.CreateFile({'id':id4}) 
downloaded4.GetContentFile('test_identity.csv')  
downloaded5 = drive.CreateFile({'id':id5}) 
downloaded5.GetContentFile('sample_submission.csv')

## 1.2. Importing train datasets

In [0]:
df_id = pd.read_csv("train_identity.csv")
df_trans = pd.read_csv("train_transaction.csv")
test_id = pd.read_csv('test_identity.csv')
test_trans = pd.read_csv('test_transaction.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [0]:
print(df_id.shape)
print(df_trans.shape)

(144233, 41)
(590540, 394)


## 1.3. Memory reduction

In [0]:
def resumetable(df):
    n = df.shape[0]
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values  
    summary['Missing %'] = round(summary['Missing'] / n * 100,2)
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    for name in summary['Name'].value_counts().index:
        summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 

    return summary

## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True, object_tranform = False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)   
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def CalcOutliers(df_num): 

    # calculating mean and std of the array
    data_mean, data_std = np.mean(df_num), np.std(df_num)

    # seting the cut line to both higher and lower values
    # You can change this value
    cut = data_std * 3

    #Calculating the higher and lower cut values
    lower, upper = data_mean - cut, data_mean + cut

    # creating an array of lower, higher and total outlier values 
    outliers_lower = [x for x in df_num if x < lower]
    outliers_higher = [x for x in df_num if x > upper]
    outliers_total = [x for x in df_num if x < lower or x > upper]

    # array without outlier values
    outliers_removed = [x for x in df_num if x > lower and x < upper]
    print('Lower bounded value: {:.2f}'.format(lower))
    print('Upper bounded value: {:.2f}'.format(upper))
    print('Identified lowest outliers: %d' % len(outliers_lower)) # printing total number of values in lower cut of outliers
    print('Identified upper outliers: %d' % len(outliers_higher)) # printing total number of values in higher cut of outliers
    print('Total outlier observations: %d' % len(outliers_total)) # printing total number of values outliers of both sides
    print('Non-outlier observations: %d' % len(outliers_removed)) # printing total number of non outlier values
    print("Total percentual of Outliers: ", round((len(outliers_total) / len(outliers_removed) )*100, 4)) # Percentual of outliers in points
    
    return

In [0]:
## REducing memory
%%time
df_trans = reduce_mem_usage(df_trans)
df_id = reduce_mem_usage(df_id)
test_trans = reduce_mem_usage(test_trans)
test_id = reduce_mem_usage(test_id)

Mem. usage decreased to 542.35 Mb (69.4% reduction)
Mem. usage decreased to 25.86 Mb (42.7% reduction)
Mem. usage decreased to 472.59 Mb (68.9% reduction)
Mem. usage decreased to 25.44 Mb (42.7% reduction)
CPU times: user 2min 54s, sys: 23.8 s, total: 3min 18s
Wall time: 3min 18s


To see the output of the Resume Table, click to see the output 

# 2. Data Engineering

In [0]:
df_train = df_trans.merge(df_id, how='left', left_index=True, right_index=True, on='TransactionID', indicator=True)
df_test = test_trans.merge(test_id, how='left', left_index=True, right_index=True, on='TransactionID', indicator = True)

df_test.rename(dict(zip(test_id.columns, df_id.columns)),inplace = True, axis = 1)

In [0]:
df_train['TransactionAmt'] = np.log(df_train.TransactionAmt)
df_test['TransactionAmt'] = np.log(df_test.TransactionAmt)


In [0]:
for col in ['card1','card2', 'card3', 'card4', 'card5', 'card6']:
    df_train[col].fillna(-99999, inplace=True)
    df_test[col].fillna(-99999, inplace=True)
    df_train[col] = df_train[col].astype('object')
    df_test[col] = df_test[col].astype('object')

In [0]:
for col in ['addr1', 'addr2']:
    df_train[col].fillna(-99999, inplace=True)
    df_test[col].fillna(-99999, inplace=True)
    df_train[col] = df_train[col].astype('object')
    df_test[col] = df_test[col].astype('object')

In [0]:
df_train['dist1_na'] = pd.isna(df_train['dist1'])
df_train['dist2_na'] = pd.isna(df_train['dist2'])
df_train['dist2'] = df_train[['dist1','dist2']].apply(lambda x: 0 if pd.isna(x['dist2']) & pd.isna(x['dist1']) == False else x['dist2'], axis = 1)
df_train['dist1'].fillna(df_train['dist1'].median(), inplace = True)

df_test['dist1_na'] = pd.isna(df_test['dist1'])
df_test['dist2_na'] = pd.isna(df_test['dist2'])
df_test['dist2'] = df_test[['dist1','dist2']].apply(lambda x: 0 if pd.isna(x['dist2']) & pd.isna(x['dist1']) == False else x['dist2'], axis = 1)
df_test['dist1'].fillna(df_test['dist1'].median(), inplace = True)

In [0]:
df_train['R_emaildomain'] = df_train[['P_emaildomain','R_emaildomain']].apply(lambda x: 'Maybe_P' if pd.isna(x['R_emaildomain']) & (pd.isna(x['P_emaildomain']) == False) else x['R_emaildomain'], axis = 1)
df_test['R_emaildomain'] = df_test[['P_emaildomain','R_emaildomain']].apply(lambda x: 'Maybe_P' if pd.isna(x['R_emaildomain']) & pd.isna(x['P_emaildomain']) == False else x['R_emaildomain'], axis = 1)

for df in [df_train, df_test]:
    df['R_emaildomain'].fillna('gmail.com', inplace=True)
    df['P_emaildomain'].fillna('gmail.com', inplace=True)

    df.loc[df['P_emaildomain'].isin(['yahoo.co.uk','yahoo.co.jp', 
                                             'yahoo.de', 'yahoo.fr','yahoo.es']),
                'P_emaildomain'] = 'yahoo.com'
    df.loc[df['P_emaildomain'].isin(['hotmail.es','hotmail.co.uk', 'hotmail.de','hotmail.fr']), 
                'P_emaildomain'] = 'hotmail.com'
    df.loc[df['P_emaildomain'].isin(['outlook.es']), 
                'P_emaildomain'] = 'outlook.com'
    df.loc[df['P_emaildomain'].isin(['outlook.es']), 
                'P_emaildomain'] = 'outlook.com'
    df.loc[df['P_emaildomain'].isin(['live.fr']), 
                'P_emaildomain'] = 'live.com'
    df.loc[df.P_emaildomain.isin(df.P_emaildomain\
                                            .value_counts()[df.P_emaildomain.value_counts() <= 300 ]\
                                            .index), 'P_emaildomain'] = "Others"
    df.loc[df['R_emaildomain'].isin(['yahoo.co.uk','yahoo.co.jp', 
                                             'yahoo.de', 'yahoo.fr','yahoo.es']),
                'R_emaildomain'] = 'yahoo.com'
    df.loc[df['R_emaildomain'].isin(['hotmail.es','hotmail.co.uk', 'hotmail.de','hotmail.fr']), 
                'R_emaildomain'] = 'hotmail.com'
    df.loc[df['R_emaildomain'].isin(['outlook.es']), 
                'R_emaildomain'] = 'outlook.com'
    df.loc[df['R_emaildomain'].isin(['outlook.es']), 
                'R_emaildomain'] = 'outlook.com'
    df.loc[df['R_emaildomain'].isin(['live.fr']), 
                'R_emaildomain'] = 'live.com'
    df.loc[df.R_emaildomain.isin(df.R_emaildomain\
                                            .value_counts()[df.R_emaildomain.value_counts() <= 110 ]\
                                            .index), 'R_emaildomain'] = "Others"

In [0]:
for col in ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8',
                      'C9', 'C10', 'C11', 'C12', 'C13', 'C14'] :
    df_test[col].fillna(df_test[col].mode()[0], inplace=True)

In [0]:
for col in ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9'] : 
    df_train[col].fillna('Miss', inplace = True)
    df_test[col].fillna('Miss', inplace = True)

In [0]:
START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")
for df in [df_train, df_test]:
    df["Date"] = df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))

    df['_Weekdays'] = df['Date'].dt.dayofweek
    df['_Hours'] = df['Date'].dt.hour
    df['_Days'] = df['Date'].dt.day
    df.drop('Date', axis =1, inplace = True)

In [0]:
for col in ['id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29']:
    df_train[col] = df_train[col].fillna('Miss')
    df_test[col] = df_test[col].fillna('Miss')
df_train.drop('id_23', inplace = True, axis = 1)
df_test.drop('id_23', inplace = True, axis =  1)

In [0]:
for df in [df_train, df_test]:
    df.loc[df['id_30'].str.contains('Windows', na=False), 'id_30'] = 'Windows'
    df.loc[df['id_30'].str.contains('iOS', na=False), 'id_30'] = 'iOS'
    df.loc[df['id_30'].str.contains('Mac OS', na=False), 'id_30'] = 'Mac'
    df.loc[df['id_30'].str.contains('Android', na=False), 'id_30'] = 'Android'
    df['id_30'].fillna("Miss", inplace=True)

In [0]:
for df in [df_train, df_test]:
    df.loc[df['id_31'].str.contains('chrome', na=False), 'id_31'] = 'Chrome'
    df.loc[df['id_31'].str.contains('firefox', na=False), 'id_31'] = 'Firefox'
    df.loc[df['id_31'].str.contains('safari', na=False), 'id_31'] = 'Safari'
    df.loc[df['id_31'].str.contains('edge', na=False), 'id_31'] = 'Edge'
    df.loc[df['id_31'].str.contains('ie', na=False), 'id_31'] = 'IE'
    df.loc[df['id_31'].str.contains('samsung', na=False), 'id_31'] = 'Samsung'
    df.loc[df['id_31'].str.contains('opera', na=False), 'id_31'] = 'Opera'
    df['id_31'].fillna("Miss", inplace=True)
    df.loc[df.id_31.isin(df.id_31.value_counts()[df.id_31.value_counts() < 200].index), 'id_31'] = "Others"
    df.loc[df.DeviceInfo.isin(df.DeviceInfo.value_counts()[df.DeviceInfo.value_counts() < 2000].index), 'DeviceInfo'] = 'Others' 

In [0]:
for col in ['id_02', 'id_03', 'id_04', 'id_05', 'id_06',
      'id_07', 'id_08', 'id_09', 'id_10', 'id_11'] : 
    if df_train[col].isnull().mean() > 0.9 :
        df_train.drop(col, inplace = True, axis = 1)
        df_test.drop(col, inplace = True, axis = 1)
    else :
        df_train[col].fillna(df_train[col].median(), inplace = True)
        df_test[col].fillna(df_test[col].median(), inplace = True)

In [0]:
for col in ['id_12', 'id_13', 'id_14',
       'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22',
       'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'] : 
    df_train[col] = df_train[col].astype('object')
    df_test[col] = df_test[col].astype('object')
    if df_train[col].isnull().mean() > 0.9 :
        df_train.drop(col, inplace = True, axis = 1)
        df_test.drop(col, inplace = True, axis = 1)
    else :
        df_train[col].fillna(df_train[col].mode()[0], inplace = True)
        df_test[col].fillna(df_test[col].mode()[0], inplace = True)

In [0]:
for col in df_train.columns.drop('isFraud') : 
    if df_train[col].isnull().mean() > 0.9 :
        df_train.drop(col, inplace = True, axis = 1)
        df_test.drop(col, inplace = True, axis = 1)
    elif df_train[col].isnull().mean() > 0 :
        df_train[col].fillna(df_train[col].median(), inplace = True)
        df_test[col].fillna(df_test[col].median(), inplace = True)

In [0]:
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

Mem. usage decreased to 654.98 Mb (5.1% reduction)
Mem. usage decreased to 569.23 Mb (5.1% reduction)


In [0]:
categorical_features = []
for col in df_train.columns.drop('isFraud') :
    if df_train[col].dtype == 'object' or df_test[col].dtype=='object':
        categorical_features.append(col)


In [0]:
resumetable(df_train[categorical_features])

Dataset Shape: (590540, 36)


,Name,dtypes,Missing,Missing %,Uniques,First Value,Second Value,Third Value,Entropy
0,ProductCD,object,0,0.0,5,W,W,W,1.28
1,card1,object,0,0.0,13553,13926,2755,4663,9.97
2,card2,object,0,0.0,501,-inf,404,490,6.34
3,card3,object,0,0.0,115,150,150,150,0.70
4,card4,object,0,0.0,5,discover,mastercard,visa,1.11
5,card5,object,0,0.0,120,142,102,166,2.70
6,card6,object,0,0.0,5,credit,credit,debit,0.84
7,addr1,object,0,0.0,333,315,325,330,5.00
8,addr2,object,0,0.0,75,87,87,87,0.58
9,P_emaildomain,object,0,0.0,31,gmail.com,gmail.com,outlook.com,2.36


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
path = '/content/gdrive/My Drive/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#df_train.to_pickle(path +'df_train_v2.pkl')
#df_test.to_pickle(path + 'df_test_v2.pkl')